In [ ]:
import torch
import torch.nn as nn
import os
import numpy as np
from PIL import Image
from torchvision import transforms
import heapq
import time
import csv

Processed ecg_record_100.png
Processed ecg_record_101.png
Processed ecg_record_102.png
Processed ecg_record_103.png
Processed ecg_record_104.png
Processed ecg_record_105.png
Processed ecg_record_106.png
Processed ecg_record_107.png
Processed ecg_record_108.png
Processed ecg_record_109.png
Processed ecg_record_111.png
Processed ecg_record_112.png
Processed ecg_record_113.png
Processed ecg_record_114.png
Processed ecg_record_115.png
Processed ecg_record_116.png
Processed ecg_record_117.png
Processed ecg_record_118.png
Processed ecg_record_119.png
Processed ecg_record_121.png
Processed ecg_record_122.png
Processed ecg_record_123.png
Processed ecg_record_124.png
Processed ecg_record_200.png
Processed ecg_record_201.png
Processed ecg_record_202.png
Processed ecg_record_203.png
Processed ecg_record_205.png
Processed ecg_record_207.png
Processed ecg_record_208.png
Processed ecg_record_209.png
Processed ecg_record_210.png
Processed ecg_record_212.png
Processed ecg_record_213.png
Processed ecg_

CAE

In [ ]:
class ECG_CAE(nn.Module):
    def __init__(self):
        super(ECG_CAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=5, stride=2, padding=2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, 32, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        _, _, h, w = x.size()
        decoded = decoded[:, :, :h, :w]
        return encoded, decoded



Huffman

In [ ]:

class node:
    def __init__(self, frequency, symbol, left=None, right=None):
        self.frequency = frequency
        self.symbol = symbol
        self.left = left
        self.right = right
        self.huffman_direction = ''

    def __lt__(self, nxt):
        return self.frequency < nxt.frequency

def get_frequency(bit_string):
    freq = {}
    for i in range(0, len(bit_string), 8):
        byte = bit_string[i:i+8]
        freq[byte] = freq.get(byte, 0) + 1
    return freq

def get_merged_huffman_tree(freq):
    heap = [node(f, b) for b, f in freq.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        left.huffman_direction = '0'
        right.huffman_direction = '1'
        merged = node(left.frequency + right.frequency, left.symbol + right.symbol, left, right)
        heapq.heappush(heap, merged)
    return heap[0]

def calculate_huffman_codes(n, code='', codes={}):
    code += n.huffman_direction
    if n.left:
        calculate_huffman_codes(n.left, code, codes)
    if n.right:
        calculate_huffman_codes(n.right, code, codes)
    if not n.left and not n.right:
        codes[n.symbol] = code
    return codes

def compress(bit_string):
    freq = get_frequency(bit_string)
    tree = get_merged_huffman_tree(freq)
    codes = calculate_huffman_codes(tree)
    compressed = ''.join(codes[bit_string[i:i+8]] for i in range(0, len(bit_string), 8))
    return compressed, codes

def decompress(compressed_string, codes):
    reverse = {v: k for k, v in codes.items()}
    current = ""
    decompressed = ""
    for bit in compressed_string:
        current += bit
        if current in reverse:
            decompressed += reverse[current]
            current = ""
    return decompressed



In [ ]:

transform = transforms.Compose([
    transforms.Resize((296, 1024)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

durations = []

def compress_and_reconstruct_folder(model, input_folder, output_folder, device):
    os.makedirs(output_folder, exist_ok=True)

    def bin16_to_signed_int(b):
        val = int(b, 2)
        if val >= 2**15:
            val -= 2**16
        return val

    for filename in os.listdir(input_folder):
        start_time = time.time()
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue

        input_path = os.path.join(input_folder, filename)
        image = Image.open(input_path).convert('L')
        img_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            encoded, _ = model(img_tensor)

        encoded_np = encoded.cpu().numpy()
        flat = np.round(encoded_np.flatten() * 1000).astype(np.int16)  # scale up to retain precision
        bit_string = ''.join(format(val & 0xFFFF, '016b') for val in flat)

        compressed_bits, codes = compress(bit_string)

        # Decompression
        decompressed_bits = decompress(compressed_bits, codes)
        flat_vals = [bin16_to_signed_int(decompressed_bits[i:i+16]) for i in range(0, len(decompressed_bits), 16)]
        reshaped = np.array(flat_vals, dtype=np.float32).reshape(encoded_np.shape) / 1000  # scale back down
        reshaped_tensor = torch.tensor(reshaped).to(device)

        with torch.no_grad():
            decoded = model.decoder(reshaped_tensor)

        decoded = decoded[:, :, :img_tensor.size(2), :img_tensor.size(3)]  # Crop back
        output_img = decoded.squeeze().cpu().numpy()
        output_img = (output_img * 0.5) + 0.5  # De-normalize
        output_img = np.clip(output_img, 0, 1)
        output_img = (output_img * 255).astype(np.uint8)

        save_path = os.path.join(output_folder, filename)
        Image.fromarray(output_img).save(save_path)

        print(f"Processed {filename}")

        end_time = time.time()
        duration = end_time - start_time
        durations.append((filename, duration))

    csv_path = 'cae_huffman.csv'
    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Image Name', 'Duration (seconds)'])
        writer.writerows(durations)
    
    print(f"✅ Duration tracking complete. Saved to: {csv_path}")



In [ ]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ECG_CAE().to(device)
    model.load_state_dict(torch.load("ecg_cae.pth", map_location=device))
    model.eval()
    compress_and_reconstruct_folder(model, "org_ecg_10sec_resize", "caehuffman", device)